In [1]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.7 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git

  Cloning https://github.com/tencent-ailab/IP-Adapter.git to /tmp/pip-req-build-8m96ufnq
  Running command git clone --filter=blob:none --quiet https://github.com/tencent-ailab/IP-Adapter.git /tmp/pip-req-build-8m96ufnq
  Resolved https://github.com/tencent-ailab/IP-Adapter.git to commit 5a18b1f3660acaf8bee8250692d6fb3548a19b14
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ip-adapter: filename=ip_adapter-0.1.0-py3-none-any.whl size=31230 sha256=e969e2e2e3b341ddb09097f727ea50e25e8756521c85801b759ae4c3003435d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-3jhrbu9p/wheels/db/b9/4a/cbddf1447388c3c9e674d4b1eb51f4661cd8ae89b90f34deee
Successfully built ip-adapter


In [3]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 950.5 kB/s eta 0:00:00


In [4]:
!cd IP-Adapter
!git lfs install
!git clone https://huggingface.co/h94/IP-Adapter


/bin/bash: line 1: cd: IP-Adapter: No such file or directory
Git LFS initialized.
Cloning into 'IP-Adapter'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 97 (delta 16), reused 0 (delta 0), pack-reused 51
Unpacking objects: 100% (97/97), 396.66 KiB | 6.10 MiB/s, done.
Filtering content: 100% (24/24), 18.41 GiB | 42.65 MiB/s, done.


In [3]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterXL

In [4]:
base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
image_encoder_path = "/content/IP-Adapter/sdxl_models/image_encoder"
ip_ckpt = "/content/IP-Adapter/sdxl_models/ip-adapter_sdxl.bin"
device = "cuda"

In [2]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [1]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter import IPAdapter

In [ ]:
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    add_watermarker=False,
)

In [ ]:
# load ip-adapter
ip_model = IPAdapterXL(pipe, image_encoder_path, ip_ckpt, device)

In [ ]:
image = Image.open("/content/assets/images/garment.png")
image.resize((512, 512))

In [ ]:
num_samples = 5
images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30, seed=420)
grid = image_grid(images, 1, num_samples)
grid